# Modelling

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import ipywidgets as widgets
from scipy.integrate import solve_ivp
from ipywidgets import interactive

def integrate(model, y0, parameters, t_end, n_points=100):
    sol = solve_ivp(
        lambda _, y, *args: model(*y, *args),
        args=parameters,
        y0=list(y0.values()),
        t_span=(0, t_end),
        t_eval=np.linspace(0, t_end, n_points),
        method="LSODA",
    )
    return pd.DataFrame(sol.y.T, index=sol.t, columns=y0.keys())


In [ ]:
def exponential_growth(N, r):
    """
    N: size of the population at time t
    r: growth rate in hours
    """
    return r * N


In [ ]:
df = integrate(
    exponential_growth,
    y0={"Population": 1.0},
    parameters=(2,),
    t_end=5,
)

df.plot(
    xlabel="Time / days",
    ylabel="Population",
    title="My fancy title",
    grid=True,
)
plt.savefig("exponential-growth.png", dpi=300)

In [ ]:
for r in [1, 0.5, 0.1]:
    df = integrate(
        exponential_growth,
        y0={"Population": 1.0},
        parameters=(r,),
        t_end=5,
    )

    df.plot(
        xlabel="Time / days",
        ylabel="Population",
        title="My fancy title",
        grid=True,
    )

In [ ]:
fig, ax = plt.subplots()
for r in [0.1, 0.5, 1]:
    df = integrate(
        exponential_growth,
        y0={"Population": 1.0},
        parameters=(r,),
        t_end=5,
    )

    df.plot(xlabel="Time / days", ylabel="Population", title="My fancy title", grid=True, ax=ax)

In [ ]:
fig, ax = plt.subplots()
rs = [0.1, 0.5, 1]
for r in rs:
    df = integrate(
        exponential_growth,
        y0={"Population": 1.0},
        parameters=(r,),
        t_end=5,
    )

    df.plot(
        xlabel="Time / days",
        ylabel="Population",
        title="My fancy title",
        grid=True,
        ax=ax,
    )
ax.legend(rs)

In [ ]:
fig, ax = plt.subplots()
for r in [0.1, 0.5, 1]:
    df = integrate(
        exponential_growth,
        y0={"Population": 1.0},
        parameters=(r,),
        t_end=5,
    )
    ax.plot(df, label=r)

ax.set_xlabel("Time / days")
ax.set_ylabel("Population")
ax.set_title("My fancy title")
ax.grid(True)
ax.legend()

In [ ]:
results = {}
for r in [0.1, 0.5, 1]:
    results[r] = integrate(
        exponential_growth,
        y0={"Population": 1.0},
        parameters=(r,),
        t_end=5,
    )


In [ ]:
dfs = pd.concat(results, axis=1)


In [ ]:
dfs.loc[3:3.04]


In [ ]:
# df = integrate(
#     exponential_growth,
#     y0={"Population": 1.0},
#     parameters=(100,),
#     t_end=10,
# )

# df.plot(
#     xlabel="Time / days",
#     ylabel="Population",
#     title="My fancy title",
#     grid=True,
# )

In [ ]:
def logistic_growth(N, r, K):
    """
    N: population size
    r: growth rate
    K: carrying capacity
    """
    return r * N * (1 - N / K)


(
    integrate(
        logistic_growth,
        y0={"Population": 1.0},
        parameters=(2, 1000),
        t_end=15,
    ).plot(
        xlabel="Time / days",
        ylabel="Population",
    )
)

# k/2

# Exercise

Change the initial condition to

- 1
- K / 2
- K
- 3 / 2 K 

and plot all curves on one axis. What do you observe?

In [ ]:
# changing the initial conditions
fig, ax = plt.subplots()
for a in [0, 1, 500, 1000, 1500]:
    df = integrate(
        logistic_growth,
        y0={"Population": a},
        parameters=(2, 1000),
        t_end=20,
    )
    ax.plot(df, label=a)

ax.set_xlabel("Time / days")
ax.set_ylabel("Population")
ax.set_title("Logistic growth")
ax.grid(True)
ax.legend()

## Lotka - Volterra

$$\large\begin{align}
    \frac{dX}{dt} &= rX - aXY \\
    \frac{dY}{dt} &= bXY - cY \\
\end{align}$$

In [ ]:
def lotka_volterra(X, Y, r, a, b, c):
    """
    this is a simple toy odel for population
    X: prey
    Y: predator
    """
    dX = r * X - a * X * Y
    dY = b * X * Y - c * Y
    return (dX, dY)


Susan = integrate(
    lotka_volterra,
    y0={"gazelles": 1.0, "lions": 1.0},
    parameters=(1, 2.5, 0.5, 2),
    t_end=100,
    n_points=10000,
)
_ = Susan.plot()


In [ ]:
Susan.plot(x="gazelles", y="lions", ylabel="lions")

In [ ]:
def lotka_volterra_with_capacity(X, Y, r, a, b, c, K):
    """
    this is a simple toy odel for population
    X: prey
    Y: predator
    """
    dX = r * X * (1  -X / K) - a * X * Y
    dY = b * X * Y - c * Y
    return (dX, dY)


Susan = integrate(
    lotka_volterra_with_capacity,
    y0={"gazelles": 1.0, "lions": 1.0},
    parameters=(1, 2.5, 0.5, 2, 10),
    t_end=100,
    n_points=10000,
)
_ = Susan.plot()


## SIR

In [ ]:
def sir(S, I, R, a, b):
    return (-a * S * I, a * S * I - b * I, b * I)


Susan = integrate(
    sir,
    y0={"S": 36.0, "I": 1.0, "R": 0},
    parameters=(0.02, 0.001),
    t_end=5000,
    n_points=10000,
)
Susan.plot(
    xlabel="time/months",
    ylabel="Human Population",
    title="A plot of a Non-communicable disease",
)

In [ ]:
def f(a, b):
    integrate(
        sir,
        y0={"S": 36.0, "I": 1.0, "R": 0},
        parameters=(a, b),
        t_end=5000,
        n_points=10000,
    ).plot()
    plt.show()


interactive_plot = interactive(
    f,
    a=widgets.FloatSlider(
        value=0.001,
        min=1e-4,
        max=1e-3,
        step=1e-4,
        readout_format=".1e",
    ),
    b=widgets.FloatSlider(
        value=0.001,
        min=1e-4,
        max=1e-3,
        step=1e-4,
        readout_format=".1e",
    ),
)
output = interactive_plot.children[-1]
output.layout.height = "350px"
interactive_plot